# Introduction
This notebook demonstrates a few example uses for pySSV.

In these first few examples we demonstrate some basic fragment shaders.

In [30]:
# Google colab support
try:
    # Try enabling custom widgets, this will fail silently if we're not in Google Colab
    from google.colab import output
    output.enable_custom_widget_manager()
    # Install pySSV for this session
    %pip install pySSV
except:
    pass

In [13]:
import pySSV as ssv
import logging
ssv.ssv_logging.set_severity(logging.INFO)

In [14]:
# Create a new SSVCanvas, the canvas is responsible for managing the OpenGL context, the render widget, and the state of the renderer.
canvas = ssv.canvas()
# Check what graphics adapter we're using
canvas.dbg_log_context()
# Set up a very basic shader program to check it's working
canvas.shader("""
#pragma SSV shadertoy mainImage
vec4 mainImage(in vec2 fragCoord)
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.yy;
    // Colour changing over time
    vec3 col = sin(uv.xyx + iTime * vec3(3, 4, 5)) * 0.5 + 0.5;
    float alpha = smoothstep(0.1, 0.1+2./iResolution.y, 1.-length(uv*2.-1.));
    // Output to screen
    return vec4(vec3(col), alpha);
}
""")

In [16]:
# run() starts the render loop, it will continuously render frames until stop() is called or the widget is destroyed.
# We set the stream mode to png here as it supports transparency. In general though, jpg (the default) is much faster.
canvas.run(stream_mode="png")

SSVRenderWidget(mouse_pos_x=331, mouse_pos_y=3, status_connection=True, status_logs='[pySSV_Render] [INFO] [ss…

In [17]:
canvas.stop()

### Mouse input
Here's a basic example of a shader that makes use of mouse position. With the `dbg_shader()` method, glsl code is generated around your shader to support ShaderToy-like shaders. In this case the canvas resolution is passed is in as `iResolution` and the mouse position as `iMouse`.

In [18]:
import pySSV as ssv
canvas1 = ssv.canvas()
canvas1.shader("""
#pragma SSV shadertoy mainImage
vec4 mainImage( in vec2 fragCoord )
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xx;
    float aaScale = 1./iResolution.x;

    vec2 mouse = uv-iMouse.xy / iResolution.xx;

    // Time varying pixel color
    vec3 col = vec3(smoothstep(0.9, .95, 1.-length(mouse)));
    col -= 1.-vec3(step(dot(step(abs(mouse), vec2(0.8/iResolution.x, 5./iResolution.x)), vec2(0.5)), 0.5));
    col -= 1.-vec3(step(dot(step(abs(mouse), vec2(5./iResolution.x, 0.8/iResolution.x)), vec2(0.5)), 0.5));

    // Output to screen
    return vec4(vec3(col), 1.0);
}
""")
canvas1.run()

SSVRenderWidget(streaming_mode='jpg')

Here's a more complex shader taken almost directly from ShaderToy.

In [19]:
canvas2 = ssv.canvas()
canvas2.shader("""
#pragma SSV shadertoy mainImage
// Copyright Thomas Mathieson all rights reserved
// https://www.shadertoy.com/view/DsffWM
const float motionBlur = 0.3;
const float aa = 0.6;
const vec3 col1 = vec3(13., 45., 140.)/100.;
const vec3 col2 = vec3(255., 20., 50.)/255.;
const vec3 col3 = vec3(21., 191., 112.)/600.;
const vec3 col4 = vec3(0.35, 1., 0.7)*0.65;
const float speed = 0.1;

float sigmoid(float x)
{
    return 1.*x/(abs(x)+1.);
}
vec3 sigmoid(vec3 x)
{
    return x/(abs(x)+vec3(1.));
}
vec3 saturate(vec3 x)
{
    return clamp(x, 0., 1.);
}
vec3 blend(float x, vec3 c)
{
    c = pow(c, vec3(x+2.));
    return mix(x*c, x*(1.-c), step(x, 0.));
}

float f(vec2 p, float t, vec4 o, vec4 o1, float s, vec4 scale)
{
    vec4 i0 = cos(t+o)*vec4(o.xw, o1.xw);
    vec4 i1 = sin(t+o1)*vec4(o.xw, o1.xw);
    vec4 x0 = i0*s*sin(scale*length(p*o.xy+4.*scale.zw)+o.z+t*o.w);
    vec4 x1 = i1*s*sin(scale*length(p*o1.xy)+o1.z+t*o1.w);
    return sigmoid(dot(x0+x1, vec4(1.)));
}

vec3 scene(float t, float emphasis, vec2 uv)
{
    // "Beautiful" randomness, tuned for aesthetics, not performance
    vec2 p = uv * 3.;
    t += 160.;
    t *= speed;
    vec4 scale = vec4(sin(t*vec3(0.25, .5, .75)), cos(t*.95))*.25+.5;
    float s0 = f(p, t, vec4(6.,9.,2.,1.5), vec4(2.,9.,7.,3.), .25, scale);
    float s1 = f(p, t, vec4(2.,6.5,1.5,4.0), vec4(3.,2.5,3.8,1.6), .5, scale);
    float s2 = sigmoid(s0/s1)*0.5;
    float s3 = f(p, t, vec4(2.,9.,7.,3.), vec4(6.,3.,2.,1.5), .125, scale);
    float s6 = f(p*1.5, t, vec4(6.,4.,8.,2.5), vec4(3.2,1.6,9.7,7.9), .25, scale);
    float s7 = f(p*1.3, t, vec4(2.,6.5,1.5,4.0), vec4(3.,2.5,3.8,1.6), .5, scale);
    float s8 = sigmoid(s6/s7+s0)*0.7;
    
    vec3 c = vec3(sigmoid((blend(s8,col1)+blend(s2,col2)+blend(s1,col3)+s7*1.)*1.1)*.7+.5);
    float grad = sigmoid(pow(length(uv*2.-1.)+s3*.3, 5.))*1.5;
    float accent = 1.-sigmoid((pow(2.5, abs(sigmoid(s8+s0+s1))-1.)-.45-(emphasis*0.1))*1000./(1.+30.*grad+20.*emphasis));
    c = mix(c, c.r*.3+col4*.8, accent);
    return clamp(vec3(c), 0., 1.);
}

vec4 mainImage( in vec2 fragCoord )
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xx;
    float aaScale = 1./iResolution.x;

    vec2 mouse = uv-iMouse.xy /iResolution.xx;
    float emp = sigmoid(1./pow(length(mouse*1.), 1.8)*.02);

    // Time varying pixel color
    vec3 col = scene(iTime, emp, uv);
    //col     += scene(iTime + motionBlur*0.001, emp, uv + aaScale*aa*vec2(0.,1.))
    //         + scene(iTime + motionBlur*0.002, emp, uv + aaScale*aa*vec2(1.,0.));
    //col /= 3.;

    // Output to screen
    return vec4(vec3(col), 1.0);
}
""")
canvas2.run(stream_quality=100)

SSVRenderWidget(streaming_mode='jpg')

Here's an example using the signed distance field template:

In [1]:
import pySSV as ssv
canvas3 = ssv.canvas()
canvas3.shader("""
#pragma SSV sdf sdf_main --camera_distance 2. --rotate_speed 1.5 --render_mode SOLID

// SDF taken from: https://iquilezles.org/articles/distfunctions/
float sdCappedTorus(vec3 p, vec2 sc, float ra, float rb) {
  p.x = abs(p.x);
  float k = (sc.y*p.x>sc.x*p.y) ? dot(p.xy,sc) : length(p.xy);
  return sqrt( dot(p,p) + ra*ra - 2.0*ra*k ) - rb;
}

float sdf_main(vec3 p) {
    float t = 2.*(sin(uTime)*0.5+0.5)+0.2;
    return sdCappedTorus(p, vec2(sin(t), cos(t)), 0.5, 0.2);
}
""")
canvas3.run(stream_quality=100)

SSVRenderWidget(streaming_mode='jpg')

In [6]:
if "canvas" in globals():
    canvas.stop()
if "canvas1" in globals():
    canvas1.stop()
if "canvas2" in globals():
    canvas2.stop()
if "canvas3" in globals():
    canvas3.stop()

### Debugging Shaders
Shaders can get quite complex so pySSV provides a few tools to simplify debugging your shaders.

#### Preprocessor Dump
It can be helpful to view the GLSL generated by the pre processor to understand why things are going wrong:

In [4]:
import pySSV as ssv
canvas1 = ssv.canvas()
shader = canvas1.dbg_preprocess_shader("""
#pragma SSV shadertoy mainImage
vec4 mainImage( in vec2 fragCoord )
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xx;
    float aaScale = 1./iResolution.x;

    vec2 mouse = uv-iMouse.xy / iResolution.xx;

    // Time varying pixel color
    vec3 col = vec3(smoothstep(0.9, .95, 1.-length(mouse)));
    col -= 1.-vec3(step(dot(step(abs(mouse), vec2(0.8/iResolution.x, 5./iResolution.x)), vec2(0.5)), 0.5));
    col -= 1.-vec3(step(dot(step(abs(mouse), vec2(5./iResolution.x, 0.8/iResolution.x)), vec2(0.5)), 0.5));

    // Output to screen
    return vec4(vec3(col), 1.0);
}
""")
print("#####################################")
print("##########  Vertex Shader  ##########")
print("#####################################")
print(shader["vertex_shader"])
print("#####################################")
print("########## Fragment Shader ##########")
print("#####################################")
print(shader["fragment_shader"])

#####################################
##########  Vertex Shader  ##########
#####################################
#version 420
#extension GL_ARB_shading_language_include : require
#line 3 "global_uniforms.glsl"
uniform float uTime;
uniform vec4 uResolution;
uniform vec2 uMouse;
#line 23 "template_shadertoy.glsl"
in vec2 in_vert;
in vec3 in_color;
out vec3 color;
out vec2 position;
void main() {
    gl_Position = vec4(in_vert, 0.0, 1.0);
    color = in_color;
    position = in_vert*0.5+0.5;
}

#####################################
########## Fragment Shader ##########
#####################################
#version 420
#extension GL_ARB_shading_language_include : require
#line 3 "global_uniforms.glsl"
uniform float uTime;
uniform vec4 uResolution;
uniform vec2 uMouse;
#line 36 "template_shadertoy.glsl"
out vec4 fragColor;
in vec3 color;
in vec2 position;
#line 3 "TEMPLATE_DATA"
vec4 mainImage( in vec2 fragCoord )
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord

#### List Shader Templates
You can also get a list of all the installed shader templates.

In [2]:
print(canvas1.dbg_query_shader_templates(additional_template_directory=None))

Found shader templates: 

	'pixel'
		Author: Thomas Mathieson
		Description: A simple full screen pixel shader.

	'sdf'
		Author: Thomas Mathieson
		Description: A shader template which allows you to render custom signed distance functions.

	'shadertoy'
		Author: Thomas Mathieson
		Description: A simple full screen pixel shader with compatibility for Shadertoy shaders.

	'vert'
		Author: Thomas Mathieson
		Description: A minimal shader template to render vertices with their vertex colours.


#### Query Shader Template Arguments
And you can query a shader template for it's arguments.

In [3]:
print(canvas1.dbg_query_shader_template("sdf"))

usage: #pragma SSV sdf [--camera_distance CAMERA_DISTANCE] [--rotate_speed ROTATE_SPEED] [--raymarch_steps RAYMARCH_STEPS]
                       [--raymarch_distance RAYMARCH_DISTANCE] [--light_dir LIGHT_DIR] [--render_mode {SOLID,DEPTH,XRAY,ISOLINES}]
                       entrypoint

A shader template which allows you to render custom signed distance functions.

positional arguments:
  entrypoint            The name of the sdf function in the shader.

options:
  --camera_distance CAMERA_DISTANCE, -c CAMERA_DISTANCE
                        The distance of the camera from the centre of the distance field. (default: 10.0)
  --rotate_speed ROTATE_SPEED, -r ROTATE_SPEED
                        The orbit speed of the camera around the SDF, in radians/second. (default: 0.1)
  --raymarch_steps RAYMARCH_STEPS
                        The number of raymarching steps to use when rendering, turn this up if the edges of surfaces look soft. (default: 128)
  --raymarch_distance RAYMARCH_DISTANCE
 

#### OpenGL Context
If you're trying to track down a driver bug or platform specific oddity, having the graphics adapter information can be helpful

In [ ]:
canvas1.dbg_log_context(full=True)

#### Frame Times
*pySSV* also provides rudimentry frame time logging to identify bottlenecks.

In [11]:
canvas1.dbg_log_frame_times(enabled=True)
# Then you just need to run the canvas
# canvas1.dbg_render_test()
# canvas1.run()

In [12]:
canvas1.stop(force=True)